<a href="https://colab.research.google.com/github/Jimin951023/AI/blob/main/16_gensim_wiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Word2Vec實作
字詞所代表的意義非常多元，在不同狀況下，會代表不同意思。要把多元意思用單一向量表示，則必須要進行word embedding的動作，也就是把高維向量降為低維向量的過程
之前介紹過，利用分散式表示法來表達字詞向量，例如PMI、SVD..統計法..等
2013年神經網路盛行後，Tomas Mikolov利用神經網路訓練方式，來獲得字詞的表達向量，獲得很棒的成果。一般認為是利用神經網路模擬人類的理解能力，獲得不錯的分布空間所得到的成果。
本範例以維基百科wiki部分資料作範例
資料來源：https://dumps.wikimedia.org/zhwiki/20220401/zhwiki-20220401-pages-articles-multistream.xml.bz2
利用結巴分詞(jieba)進行斷詞，gensim套件進行word2vec計算
本範例約需1小時長時間執行

In [2]:
# ! wget https://dumps.wikimedia.org/zhwiki/20220401/zhwiki-20220401-pages-articles-multistream.xml.bz2

In [3]:
!wget https://dumps.wikimedia.org/zhwiki/20220401/zhwiki-20220401-pages-articles-multistream1.xml-p1p187712.bz2

--2022-04-20 12:42:40--  https://dumps.wikimedia.org/zhwiki/20220401/zhwiki-20220401-pages-articles-multistream1.xml-p1p187712.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.7, 2620:0:861:1:208:80:154:7
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 215920539 (206M) [application/octet-stream]
Saving to: ‘zhwiki-20220401-pages-articles-multistream1.xml-p1p187712.bz2’

zhwiki-20220401-pag 100%[===================>] 205.92M  4.76MB/s    in 43s     

2022-04-20 12:43:23 (4.75 MB/s) - ‘zhwiki-20220401-pages-articles-multistream1.xml-p1p187712.bz2’ saved [215920539/215920539]



In [5]:
!pip install opencc-python-reimplemented

In [7]:
from gensim.corpora import WikiCorpus

wiki_corpus = WikiCorpus('zhwiki-20220401-pages-articles-multistream1.xml-p1p187712.bz2', dictionary={})

In [9]:
wiki_corpus

In [13]:
next(iter(wiki_corpus.get_texts()))[:10]

['歐幾里得',
 '西元前三世紀的古希臘數學家',
 '現在被認為是幾何之父',
 '此畫為拉斐爾的作品',
 '雅典學院',
 '数学',
 '是研究數量',
 '从某种角度看屬於形式科學的一種',
 '數學利用抽象化和邏輯推理',
 '從計數']

In [14]:
text_num = 0

with open('wiki_text.txt', 'w', encoding='utf-8') as f:
    for text in wiki_corpus.get_texts():
        f.write(' '.join(text)+'\n')
        text_num += 1
        if text_num % 10000 == 0:
            print('{} articles processed.'.format(text_num))

    print('{} articles processed.'.format(text_num))

8948 articles processed.


In [16]:
import jieba
from opencc import OpenCC


# Initial
cc = OpenCC('s2t')
train_data = open('wiki_text.txt', 'r', encoding='utf-8').read()
train_data = cc.convert(train_data)
train_data = jieba.lcut(train_data)
train_data = [word for word in train_data if word != '']
train_data = ' '.join(train_data)
open('seg.txt', 'w', encoding='utf-8').write(train_data)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.002 seconds.
Prefix dict has been built successfully.


45133839

In [17]:
from gensim.models import word2vec


# Settings
seed = 666
sg = 0
window_size = 10
vector_size = 100
min_count = 1
workers = 8
epochs = 5
batch_words = 10000

train_data = word2vec.LineSentence('seg.txt')
model = word2vec.Word2Vec(
    train_data,
    min_count=min_count,
    size=vector_size,
    workers=workers,
    iter=epochs,
    window=window_size,
    sg=sg,
    seed=seed,
    batch_words=batch_words
)

model.save('word2vec.model')

In [18]:
from gensim.models import word2vec

string = '電腦'
model = word2vec.Word2Vec.load('word2vec.model')
print(string)

for item in model.wv.most_similar(string):
    print(item)

電腦
('ibm', 0.8495076298713684)
('手機', 0.8404494524002075)
('廠商', 0.8402011394500732)
('軟體', 0.8361618518829346)
('計算機', 0.8300386071205139)
('模擬器', 0.8217790126800537)
('智能', 0.8212008476257324)
('程式', 0.8131803870201111)
('晶片', 0.8126602172851562)
('微電腦', 0.808303952217102)
